In [1]:
from neurodiffeq import diff
from neurodiffeq.networks import FCNN
from neurodiffeq.pde import solve2D, ExampleGenerator2D, Monitor2D
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# Homogeneous Neumann B.C on a Circular Boundary

In [2]:
def _nn_output(net, xyz_etc, ith):
    original_shape  = xyz_etc[0].shape
    xyz_etc = torch.cat(xyz_etc, 1)
    output = net(xyz_etc)
    return output[:, ith].reshape(original_shape)

In [3]:
class CircularNeumannHomogeneous:
    
    def __init__(self, r, val=0.0, ith=0, K=5, alpha=5):
        def g(dimensions):
            return val
        def L_D(dimensions):
            return 1.0  # no Dirichlet bouondary condition => every where is far away from the Dirichlet boundary
        def L_M(dimensions):
            dimensions = [d.reshape(-1, 1) for d in dimensions]
            return r**2 - sum(d**2 for d in dimensions)
        def A_D(dimensions):
            return 0.0  # no Dirichlet bouondary condition => A_D doesn't really matter
        def normal_vector(dimensions):
            dimensions = [d.reshape(-1, 1) for d in dimensions]
            return dimensions
        def A_M(dimensions, net, F):
            # TODO avoid redundant calculation
            
            dimensions = [d.reshape(-1, 1) for d in dimensions]
            numer = g(dimensions) - torch.sum(
                torch.cat(normal_vector(dimensions), 1) * \
                torch.cat( tuple(diff(F, d) for d in dimensions), 1 ), 
                dim=1
            ).reshape(F.shape) # diff(A_D, d) is omitted here because it's always 0.0
            denom = L_D(dimensions) * torch.sum(
                torch.cat(normal_vector(dimensions), 1) * \
                torch.cat( tuple(diff(L_M(dimensions), d) for d in dimensions), 1 ), 
                dim=1
            ).reshape(F.shape) + K * (1-torch.exp(-alpha * L_M(dimensions)))
            return L_D(dimensions) * L_M(dimensions) * numer / denom
        
        self.L_D = L_D
        self.L_M = L_M
        self.A_D = A_D
        self.A_M = A_M
        self.ith = ith
        
    def enforce(self, net, *dimensions):
        F = self.L_D(dimensions) * _nn_output(net, dimensions, self.ith)
        return self.A_D(dimensions) + self.A_M(dimensions, net, F) + F
    
    def set_impose_on(self, ith):
        self.ith = ith

Poisson in circular domain where

$$
\partial_x^2 u + \partial_y^2 u = 1
$$

$$
x^2 + y^2 \leq 1
$$

s.t.

$$
\hat n((x, y)) \cdot \nabla u = \frac{1}{2}
$$

Solution is

$$
u(x, y) = \frac{1}{4}(x^2 + y^2) - \frac{1}{4}
$$

In [4]:
def solution_analytical_poisson(xx, yy):
    return 0.25 * (xx**2 + yy**2) - 0.25

In [5]:
%matplotlib notebook
poisson = lambda u, x, y: diff(u, x, order=2) + diff(u, y, order=2) - 1
bc  = CircularNeumannHomogeneous(r=1, val=0.5)
net = FCNN(n_input_units=2, n_hidden_units=32, n_hidden_layers=1, actv=nn.Softplus)

solution_neural_net_poisson, _ = solve2D(
    pde=poisson, condition=bc, xy_min=(-1, -1), xy_max=(1, 1),
    net=net, max_epochs=10, train_generator=ExampleGenerator2D(
        (32, 32), (-1, -1), (1, 1), method='equally-spaced-noisy'
    ),
    monitor=Monitor2D(check_every=10, xy_min=(-1, -1), xy_max=(1, 1))
)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.